## Brute force analytics

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import datetime as dt

In [4]:
from collections import Counter

In [5]:
import matplotlib.pyplot as plt

In [6]:
from IPython.display import clear_output

### Import Data

In [9]:
XRP_Increase = pd.read_csv('../XRP_5min_Increase_SmallAmount_5min.csv')
XRP_Decrease = pd.read_csv('../XRP_5min_Decrease_SmallAmount_5min.csv')

In [10]:
original_columns = ['datetime_real','date','close','high','low','open','quoteVolume','volume','weightedAverage']

## Adjust for waiting

In [11]:
XRP_Analysis_table = XRP_Increase[original_columns]

In [12]:
for col in ['Unnamed: 0','Unnamed: 0.1']:
    try:
        del XRP_Increase[col]
    except:
        None

In [13]:
XRP_Analysis_table['20min_RA_on_Close'] = XRP_Analysis_table['close'].rolling(4).mean().shift(1)
XRP_Analysis_table['1H_RA_on_Close'] = XRP_Analysis_table['close'].rolling(12).mean().shift(1)
XRP_Analysis_table['2H_RA_on_Close'] = XRP_Analysis_table['close'].rolling(24).mean().shift(1)
XRP_Analysis_table['4H_RA_on_Close'] = XRP_Analysis_table['close'].rolling(48).mean().shift(1)
XRP_Analysis_table['12H_RA_on_Close'] = XRP_Analysis_table['close'].rolling(144).mean().shift(1)

c:\users\zhen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\zhen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\zhen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [14]:
ra_list            =    ['20min_RA_on_Close', '1H_RA_on_Close', '2H_RA_on_Close', '4H_RA_on_Close', '12H_RA_on_Close']
buy_in_list        =    np.arange(0.002,0.022, 0.002)
stop_loss_list     =    np.arange(0.01,0.03, 0.002)
time_waiting_list  =    [2, 5, 8, 12, 16, 24] # in hours
sell_limit_list    =    np.arange(0.002,0.032, 0.002)


In [15]:
dates_cors = {  1507374600: 1511274300,
                1512774600: 1515174000,
                1515174300: 1518174000,
                1547873100: 1550872800}

In [16]:
for time in dates_cors.items():
    print(time[0], time[1])

1507374600 1511274300
1512774600 1515174000
1515174300 1518174000
1547873100 1550872800


In [17]:
event_dictionary = [(   'buy_in',
                        'stop_loss',
                        'sell_limit',
                        'time_waiting',
                        'ra_column',
                        'time1',
                        'time2',
                        'buyin',
                        'sell_profit',
                        'stop_loss_hit',
                        'time_stop',
                        'final amount')]

## Getting pkl files

## Failed!, the time waiting factor is not rightly recorded

In [28]:
part_1_brute = pd.read_pickle('save_xrp_brute.pkl')

In [27]:
part_2_brute = pd.read_pickle('reverse_save_xrp_brute.pkl')

In [40]:
a = part_1_brute.tail()

In [41]:
b = part_2_brute[(part_2_brute['buy_in'] == 0.012) &
             (part_2_brute['stop_loss'] == 0.018000000000000002) & 
             (part_2_brute['sell_limit'] <= 0.002) &
             (part_2_brute['time_waiting'] == '20min_RA_on_Close')]

In [53]:
len(pd.concat([part_1_brute,part_2_brute]))

190747

In [51]:
xrp_brute = pd.concat([part_1_brute,part_2_brute]).drop_duplicates()

In [56]:
xrp_brute['ra_column'].unique()

array([1507374600, 1512774600, 1515174300, 1547873100], dtype=object)

In [66]:
xrp_brute['time'].unique()

array([1511274300, 1515174000, 1518174000, 1550872800], dtype=object)

In [57]:
rename_section = {1507374600: 'section 1', 1512774600: 'section 2', 1515174300: 'section 3', 1547873100: 'section 4'}

In [69]:
xrp_brute['section'] = [rename_section[d] for d in xrp_brute['ra_column']]

In [71]:
xrp_brute['time_waiting'].unique()

array(['20min_RA_on_Close', '1H_RA_on_Close', '2H_RA_on_Close',
       '4H_RA_on_Close', '12H_RA_on_Close'], dtype=object)

In [73]:
get_time_waiting = {'20min_RA_on_Close': 0.33,
                    '1H_RA_on_Close'   : 1,
                    '2H_RA_on_Close'   : 2, 
                    '4H_RA_on_Close'   : 4,
                    '12H_RA_on_Close'  : 12 }

In [74]:
xrp_brute['time_waiting_hour'] = [get_time_waiting[d] for d in xrp_brute['time_waiting']]

In [76]:
xrp_brute_clean = xrp_brute[['buy_in','stop_loss','sell_limit','buyin','sell_profit','stop_loss_hit','time_stop','final amount','section','time_waiting_hour']].copy()

In [107]:
for col in xrp_brute_clean.columns:
    if col != 'section':
        xrp_brute_clean[col] = pd.to_numeric(xrp_brute_clean[col])

In [109]:
xrp_brute_clean.groupby('section').agg({'final amount' :'max'})

,final amount
section,
section 1,4.114028e+04
section 2,2.350654e+09
section 3,1.961909e+08
section 4,6.280000e+01


In [110]:
xrp_brute_clean.groupby('section').agg({'final amount' :'max'})['final amount'].values

array([4.11402800e+04, 2.35065361e+09, 1.96190886e+08, 6.28000000e+01])

In [111]:
xrp_brute_clean[ xrp_brute_clean['final amount'].isin(xrp_brute_clean.groupby('section').agg({'final amount' :'max'})['final amount'].values)]

,buy_in,stop_loss,sell_limit,buyin,sell_profit,stop_loss_hit,time_stop,final amount,section,time_waiting_hour
16201,0.002,0.028,0.002,3006,2927,33,45,4.114028e+04,section 1,0.33
16202,0.002,0.028,0.002,2520,2287,226,6,2.350654e+09,section 2,0.33
16203,0.002,0.028,0.002,2700,2455,199,46,1.961909e+08,section 3,0.33
34853,0.004,0.028,0.012,571,273,268,29,6.280000e+01,section 1,4.00
60065,0.008,0.016,0.012,620,309,301,9,6.280000e+01,section 1,1.00
83244,0.010,0.022,0.008,46,43,1,2,6.280000e+01,section 4,0.33
85044,0.010,0.024,0.008,46,43,1,2,6.280000e+01,section 4,0.33
86844,0.010,0.026,0.008,46,43,1,2,6.280000e+01,section 4,0.33
36710,0.016,0.028,0.020,768,239,520,9,6.280000e+01,section 2,2.00
55145,0.014,0.028,0.012,456,185,265,6,6.280000e+01,section 1,4.00


In [139]:
xrp_brute_clean.groupby(['section','buy_in','time_waiting_hour','sell_limit']).agg({'final amount':['mean', 'max']}).round(1)#.unstack(1)

final amount         
                                                      mean      max
section   buy_in time_waiting_hour sell_limit                      
section 1 0.002  0.33              0.002           10988.6  41140.3
                                   0.004            2226.0  10443.4
                                   0.006             581.9   2727.6
                                   0.008             235.7    736.9
                                   0.010             124.5    298.4
                                   0.012              98.5    236.3
                                   0.014              79.7    183.2
                                   0.016              72.3    145.6
                                   0.018              70.8    144.6
                                   0.020              63.7    139.1
                                   0.022              59.1    119.2
                                   0.024              55.6    114.4
                                   0.026              48.5     93.2
                                   0.028              48.6    100.6
                                   0.030              48.8    108.7
                 1.00              0.002            4483.7  15597.2
                                   0.004            1451.5   6252.3
                                   0.006             364.6   1778.8
                                   0.008             162.1    609.8
                                   0.010             103.9    357.2
                                   0.012              73.7    239.4
                                   0.014              52.5    155.4
                                   0.016              51.2    143.6
                                   0.018              41.4    111.8
                                   0.020              36.3    100.0
                                   0.022              36.2     99.6
                                   0.024              35.2     88.8
                                   0.026              30.7     81.4
                                   0.028              27.6     73.4
                                   0.030              26.1     68.6
...                                                    ...      ...
section 4 0.020  4.00              0.002              40.5     44.4
                                   0.004              42.5     45.4
                                   0.006              41.8     44.7
                                   0.008              41.9     44.2
                                   0.010              42.2     45.0
                                   0.012              41.8     45.0
                                   0.014              42.0     45.0
                                   0.016              42.1     45.3
                                   0.018              41.1     45.9
                                   0.020              40.5     45.2
                                   0.022              39.9     43.5
                                   0.024              40.1     44.1
                                   0.026              40.1     44.7
                                   0.028              40.2     44.9
                                   0.030              40.0     45.0
                 12.00             0.002              21.4     29.0
                                   0.004              21.5     30.3
                                   0.006              24.2     31.7
                                   0.008              23.9     32.3
                                   0.010              24.2     33.0
                                   0.012              23.7     33.0
                                   0.014              23.7     34.1
                                   0.016              23.8     34.6
                                   0.018              23.6     34.5
                                   0.020              23.7     34.9
                                  

In [ ]:
xrp_brute_clean.groupby(['section','buy_in','stop_loss']).agg({'final amount':['mean', 'max']}).round(1).unstack()

In [136]:
xrp_brute_clean.groupby(['section','buy_in','stop_loss']).agg({'final amount':['mean', 'max']}).round(1).unstack()

final amount                                                \
                         mean                                                 
stop_loss               0.010   0.012    0.014    0.016     0.018     0.020   
section   buy_in                                                              
section 1 0.002          23.2    62.2    137.7    207.0     299.5     377.9   
          0.004          15.5    36.3     86.9    138.8     201.5     293.6   
          0.006          10.0    21.9     48.0     78.6     117.2     165.4   
          0.008           8.4    20.7     42.5     61.0      88.7     120.4   
          0.010           8.9    16.6     28.9     40.2      57.0      76.5   
          0.012           9.6    11.7     23.0     33.5      45.6      57.8   
          0.014          14.6    14.8     18.1     27.2      38.8      50.2   
          0.016          20.3    21.0     22.6     29.7      41.4      48.8   
          0.018          16.6    19.4     22.0     25.1      31.6      41.5   
          0.020          18.3    19.1     19.9     29.9      30.1      35.7   
section 2 0.002         336.9  2329.8  11966.6  57637.7  188867.7  658179.6   
          0.004         138.1   736.8   3978.6  14433.1   35213.4  134094.1   
          0.006          80.9   394.2   1342.2   4571.3   14145.8   45453.5   
          0.008          30.5   115.9    414.9   1153.7    3225.3    8617.9   
          0.010          20.3    62.3    184.8    437.5    1018.3    2576.1   
          0.012          15.2    38.3     90.2    158.5     299.0     913.2   
          0.014          16.9    28.2     83.1    121.8     210.8     350.0   
          0.016          12.9    21.2     45.1     56.9      82.4     120.4   
          0.018          13.4    22.7     34.3     63.8      64.7     111.2   
          0.020          11.6    16.4     22.5     37.6      58.5      92.1   
section 3 0.002          24.5   230.6   2210.3  16986.4   51350.4  126113.6   
          0.004          10.8    85.2    852.7   6483.0   18849.0   52132.6   
          0.006           3.5    19.9    156.4   1198.7    3795.3   12238.9   
          0.008           2.0     6.9     61.5    288.8     849.7    2464.7   
          0.010           1.4     5.6     22.1    157.4     510.4    1312.2   
          0.012           1.3     4.1     15.2     46.6     118.2     356.1   
          0.014           0.7     1.3      6.1     13.6      35.9      78.1   
          0.016           0.8     1.6      3.9      9.0      16.0      37.7   
          0.018           1.6     2.6      4.9     10.7      21.4      36.2   
          0.020           2.3     3.2      5.6      9.3      16.0      26.6   
section 4 0.002          18.3    23.7     26.7     28.6      31.3      32.2   
          0.004          20.6    25.8     28.9     30.6      33.6      34.6   
          0.006          21.6    26.9     29.8     31.7      35.1      36.1   
          0.008          23.7    28.7     31.3     33.2      36.6      37.8   
          0.010          26.1    30.0     32.3     34.4      38.0      38.9   
          0.012          30.6    30.6     31.4     34.0      37.1      38.3   
          0.014          38.4    37.3     33.8     36.0      38.3      39.2   
          0.016          39.2    41.7     37.7     37.2      39.0      40.2   
          0.018          37.8    38.1     38.7     39.2      38.5      40.4   
          0.020          41.1    42.2     41.6     40.4      41.6      40.8   

                                                                         \
                                                                    max   
stop_loss             0.022      0.024       0.026       0.028    0.010   
section   buy_in                                                          
section 1 0.002       477.4      517.9       586.8       683.0    909.1   
          0.004       390.3      447.3       519.2       594.5    413.4   
          0.006       205.8      225.9       278.1       324.8    152.3   
          0.008

In [8]:
def plotting_view(xrp_data_input , trading_log ):

#     ploting_breakdown = XRP_Analysis_table[(XRP_Analysis_table['date'] >= 1507374600 ) &
#                                            (XRP_Analysis_table['date'] <= 1511274300 )]


    transaction_display = trading_log
    transaction_display['time_start'] = transaction_display.index
    transaction_display['time_end'] = round(transaction_display['time_start'].shift(-1), 0)

    transaction_display['action_to_color'] = transaction_display['action'].shift(-1)

    transaction_display['colour_section'] = transaction_display['action_to_color'].apply(lambda x: 'r' if x == 'stop_loss_hit' else 'g')
    
    ploting_breakdown = xrp_data_input
    plt.figure(figsize= (20,10))


    tran_y_low = ploting_breakdown['low']
    tran_y_high = ploting_breakdown['high']

    tran_y_ra = ploting_breakdown['2H_RA_on_Close']

    tran_x = ploting_breakdown['date']


    plt.plot(tran_x, tran_y_low)
    plt.plot(tran_x, tran_y_high)

    plt.plot(tran_x, tran_y_ra)

    graph_min = ploting_breakdown['low'].min() * 0.9
    graph_max = ploting_breakdown['high'].max() * 1.1

    # plotting transaction
    for ind, value in transaction_display[ (transaction_display['time_start'] <= ploting_breakdown['date'].max()) &
                                           (transaction_display['time_start'] >= ploting_breakdown['date'].min()) &                      
                                           (transaction_display['action'] == 'brought')].iterrows():

        plt.axvspan(value['time_start'], value['time_end'], alpha = 0.1 , facecolor = value['colour_section'])

    #     plt.vlines(value['time_start'], graph_min, graph_max, alpha = 0.4, color = value['colour_section'])
    #     plt.vlines(value['time_end'], graph_min, graph_max, alpha = 0.4, color = value['colour_section'], linestyle = 'dashed')


    # for ind, value in ploting_breakdown[ploting_breakdown['Increase_0.5_percent'] <= ploting_breakdown['date'].max()].iterrows():
    # #     print(value['Increase_0.5_percent'])
    # #     plt.vlines(value['date'], 0, 1, alpha = 0.2, color = 'b',linestyle = 'dashed')
    #     plt.vlines(value['Increase_0.5_percent'], 0, 1, alpha = 0.2, color = 'b',linestyle = 'dashed')


    plt.ylim(graph_min, graph_max )

    plt.xlim(ploting_breakdown['date'].min(), ploting_breakdown['date'].max())
    plt.legend()